## Couzin's Model
#### Parameters: 
- n : number of agents
- V : direction vectors of n agents
- C : position vectors of n agents
- tau: time step size
- r_r: radius of repulsion
- r_o: radius of orientation
- r_a: radius of attraction
- sigma: random error in direction
- theta: turning rate
- S : individual speed
- alpha: cone angle
    

In [1]:
import random
import numpy as np
import math
import plotly.graph_objects as go
import pandas as pd

In [2]:
# 1. Initialize n agents at n random locations in a sphere
# 2. Initialize n agent direction vectors

def gen_pos(n, r):
    
    # returns a random position vector corresponding to each agent
    
    C = {}
    for i in range(n):
        radius = random.uniform(0,r)
        x1 = np.random.randn()
        x2 = np.random.randn()
        x3 = np.random.randn()
    
        mag = math.sqrt((x1*x1) + (x2*x2) + (x3*x3))
        
        x1 /= mag
        x2 /= mag
        x3 /= mag
        
        radius = radius **(1./3.) 
        x1*= radius
        x2*= radius
        x3*= radius
        C[i] = np.asarray([x1,x2,x3])
    
    return C

def gen_direction(n):
    
    # returns a random direction vector corresponding to each agent
    
    V = {}
    for i in range(n):
        u = np.random.randn()
        v = np.random.randn()
        w = np.random.randn()
        mag = math.sqrt(u*u + v*v + w*w)
        
        u = u/mag
        v = v/mag
        w = w/mag
        V[i] = np.asarray([u, v, w])
        
    return V

def angle_diff(i, V, C):
    
    # returns the angle between the direction vector of each agent with the agent(i)
    
    ang_diff = []
    n = len(V)
    
    for j in range(n):
        cosang = np.dot(V[i], V[j])
        sinang = np.linalg.norm(np.cross(V[i], V[j]))
        ang_diff.append(np.arctan2(sinang,cosang))
    
    return np.degrees(ang_diff)

def distance_diff(i, V, C):
    
    # returns the distance between the position vector of each agent with the agent(i)
    
    dist_diff = []
    n = len(C)
    for j in range(n):
        
        diff = V[i] - V[j] 
        dist = np.linalg.norm(diff)
        dist_diff.append(dist)
        
    return dist_diff

def not_inside_blind_cone(point_loc, agent_loc, agent_orient, cone_height, alpha ):
    
    # Checks if the location of given agent is inside the blind cone of provided agent
    
    cone_dist = np.dot(point_loc - agent_loc, agent_orient)
    
    if cone_dist <= 0 or cone_dist > cone_height:
        return True
    
    sinang = np.linalg.norm(np.cross(point_loc-agent_loc, agent_orient))
    cosang = np.dot(point_loc-agent_loc, agent_orient)
    angle = np.degrees(np.arctan2(sinang, cosang))
    
    if angle > alpha:
        return True
    return False
    
def compute_orientation(i, distances, angles, r_r, r_o, alpha, V, C):
    
    # returns information corresponding to agents in the orientation sphere of agent i
    
    n = len(distances)
    indices = []
    
    for j in range(n):
        
        if j != i:
            if distances[j]> r_r and distances[j] <= r_o and not_inside_blind_cone(C[j],C[i],(-1 * V[i]),r_o,(360-alpha)):
                indices.append[j]
    n_o = len(indices)
    V_o = []
    C_o = []
    
    for j in range(n_o):
        V_o.append(V[indices[j]])
        C_o.append(C[indices[j]])
    
    return n_o, V_o, C_o

def compute_attraction(i, distances, angles, r_o, r_a, alpha, C):
    
    # returns information corresponding to agents in the attraction sphere of agent i
    
    n = len(distances)
    indices = []
    
    for j in range(n):
        
        if j != i and distances[j]> r_o and distances[j] <= r_a and not_inside_blind_cone(C[j],C[i],(-1 * V[i]),r_o,(360-alpha)):
            indices.append[j]
    
    n_a = len(indices)
    V_a = []
    C_a = []
    
    for j in range(n_a):
        V_a.append(V[indices[j]])
        C_a.append(C[indices[j]])
    
    return n_a, V_a, C_a
    

def compute_repulsion(i, distances, angles, r_r, V, C):
    
    # returns information corresponding to agents in the repulsion sphere of agent i
    
    n = len(distances)
    indices = []
    
    for j in range(n):
        if j != i and distances[j] <= r_r:
            indices.append(j)
            
    n_r = len(indices)
    V_r = []
    C_r = []
    
    for j in range(n_r):
        V_r.append(V[indices[j]])
        C_r.append(C[indices[j]])
    
    return n_r, V_r, C_r
        


def calculate_angle_between(old_di, di):
    
    # return angle in degrees between the old and new direction vector of agent
    
    cosang = np.dot(old_di, di)
    sinang = np.linalg.norm(np.cross(old_di, di))

    angle = np.arctan2(sinang, cosang)
    
    return np.degrees(angle)

def compute_do(n_o, V_o, C_o):
    resultant_vector = np.zeros((3,1))

    for vector in V_o:
        resultant_vector += vector/np.linalg.norm(vector)
    
    return resultant_vector

def compute_da(n_a, V_a, C_a, C_i):
    resultant_vector = np.zeros((3,1))

    for vector in C_a:
        vector_in_direction_of_j = (vector-C_i)/np.linalg.norm(vector-C_i)
        resultant_vector += vector_in_direction_of_j/np.linalg.norm(vector_in_direction_of_j)
    
    return resultant_vector

def compute_dr(C_r, C_i):
    resultant_vector = np.zeros((3,))

    for vector in C_r:
        vector_in_direction_of_j = (vector-C_i)/np.linalg.norm(vector-C_i)
        resultant_vector -= vector_in_direction_of_j/np.linalg.norm(vector_in_direction_of_j)
    
    return resultant_vector
    
    

In [3]:
# function to compute the updated values at next timestep.
def update(n, r, V, C, r_r, r_o, r_a, theta, alpha, sigma, tau, S):
    directions = {}
    positions = {}
    for i in range(n):
        # compute for each agent
        
        angles, distances = angle_diff(i, V, C), distance_diff(i, V, C)
        
        n_r, V_r, C_r = compute_repulsion(i, distances, angles, r_r, V, C)
        
        if n_r > 0:
            di = compute_dr(C_r, C[i])
        
        elif n_r == 0:
            
            n_o, V_o, C_o = compute_orientation(i, distances, angles, r_r, r_o, alpha, V, C)
            n_a, V_a, C_a = compute_attraction(i, distances, angles, r_o, r_a, alpha, C)

            do = compute_do(n_o, V_o, C_o)            
            da = compute_da(n_a, V_a, C_a, C[i])
            
            if n_o == 0 and n_a != 0 :
                di = da 
                
            elif n_a == 0 and n_o != 0:
                di = do
                
            elif n_o != 0 and n_a != 0:
                di = (do + da)*0.5

            else:
                di = V[i]
                
        # add error component in di, from gaussian with standard deviation sigma
        # err  = compute_dev(sigma)
        turn_angle = calculate_angle_between(V[i],di) 
        
        if turn_angle < (theta * tau):
            time_required_to_turn = turn_angle/theta
            remaining_time = tau - time_required_to_turn
            directions[i] = di
            positions[i] = (C[i] + di * S * remaining_time)
            
        else:
            new_di = V[i] + (di - V[i])*(theta*tau)
            directions[i] = new_di*(1/np.linalg.norm(new_di))
            positions[i] = C[i]
        
    return directions, positions


In [25]:
def val_sim(n, r, r_r, r_o, r_a, theta, alpha, sigma, tau, S, n_timesteps):
    # returns direction, location of n agents after n_timesteps
    C = gen_pos(n, r)
    V = gen_direction(n)
    
    for i in range(n_timesteps):
        V, C = update(n, r, V, C, r_r, r_o, r_a, theta, alpha, sigma, tau, S)
    return V,C

def main():
    # set values for variables
    #n = 10           # vary between 10 and 100
    #r = 20*20*20         # 
    #V = gen_direction(n, r)
    #C = gen_pos(n, r)
    #r_r = 1
    #r_o = 0 #vary between(0-15)
    #r_a = 0 #vary between(0-15)
    
    #theta = 10 # turning rate (degrees per second), vary between 10-100
    #alpha = 200 # cone angle (field of preception), vary between 200-360
    
    #sigma = 0 # random error introduction in desired direction, vary between 0-11.5 degrees
    
    #tau = 0.1 #timestep(in seconds)
    #S = [1 for i in range(n)]# individual speeds - vector of size n, vary between 1-5 
    
    # call val_sim() for different variable values ( 4 cases)
    # display the simulated plot for the 4 scenarios
    V,C = val_sim(50, 8000, 1, 10, 0, 10, 300, 0, 0.1, 1, 200)

    return V,C
    

In [26]:
if __name__ == '__main__':
    

    V, C = main()
    df = pd.DataFrame(columns=['x','y','z','u','v','w'])
    
    for i in range(len(V)):
        df.loc[i,'x'] = C[i][0] 
        df.loc[i,'y'] = C[i][1]
        df.loc[i,'z'] = C[i][2]
        df.loc[i,'u'] = V[i][0]
        df.loc[i,'v'] = V[i][1]
        df.loc[i,'w'] = V[i][2]       

    fig = go.Figure(data = go.Cone(
        x=df['x'],
        y=df['y'],
        z=df['z'],
        u=df['u'],
        v=df['v'],
        w=df['w'],
       
        
        sizeref=1))

    #fig.update_layout(scene=dict(aspectratio=dict(x=1, y=1, z=0.8),
                                 #camera_eye=dict(x=1.2, y=1.2, z=0.6)))

    fig.show()